loading data set using PyTorch

In [ ]:
import os
import json
import matplotlib.pyplot as plt
import cv2
import numpy as np
from torch.utils.data import Dataset, DataLoader

class ImageLabelDataset(Dataset):
    def __init__(self, root_dir, data_types):
        self.data = {}
        for data_type in data_types:
            data_path = os.path.join(root_dir, data_type)
            image_folder_path = os.path.join(data_path, "images")
            label_folder_path = os.path.join(data_path, "labels", "json")
            images = []
            labels = []
            for image_file in os.listdir(image_folder_path):
                image_path = os.path.join(image_folder_path, image_file)
                json_file = image_file.split('.')[0] + '.json'
                label_path = os.path.join(label_folder_path, json_file)
                if os.path.exists(label_path):
                    images.append(image_path)
                    labels.append(label_path)
            self.data[data_type] = {"images": images, "labels": labels}

    def __len__(self):
        total_images = 0
        for data_type in self.data:
            total_images += len(self.data[data_type]["images"])
        return total_images

    def __getitem__(self, idx):
     data_type = None
     image_idx = None
     for dt, data in self.data.items():
        if idx < len(data['images']):
            data_type = dt
            image_idx = idx
            break
        else:
            idx -= len(data['images'])
     if data_type is not None and image_idx is not None:
        image_path = self.data[data_type]["images"][image_idx]
        label_path = self.data[data_type]["labels"][image_idx]
        return image_path, label_path
     else:
        raise IndexError("Index out of range.")


2-visualizing some of the labeled images

In [ ]:
def visualize_images(data_loader, num_batches):
    colors = {"bin": (255, 0, 0), "dolly": (0, 255, 0), "jack": (0, 0, 255)}
    
    for batch_idx, batch in enumerate(data_loader):
        if batch_idx >= num_batches:
            break
        
        images, labels = batch
        for image_path, label_path in zip(images, labels):
            image_cv2 = cv2.imread(image_path)
            image_cv2 = cv2.cvtColor(image_cv2, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(8, 6))
            plt.imshow(image_cv2)

            with open(label_path, 'r') as f:
                labels = json.load(f)

            for bbox in labels:
                left = bbox["Left"]
                top = bbox["Top"]
                right = bbox["Right"]
                bottom = bbox["Bottom"]
                class_name = bbox["ObjectClassName"]
                color = colors.get(class_name, (0, 0, 0))
            

                cv2.rectangle(image_cv2, (left, top), (right, bottom), color, 2)
                cv2.putText(image_cv2, class_name, (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 4)

            plt.imshow(image_cv2)
            plt.title(image_path)
            plt.show()

root_dir = "data"
dataset_train = ImageLabelDataset(root_dir, ["Training"])
dataset_test = ImageLabelDataset(root_dir, ["Testing"])
batch_size = 2

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

num_batches_to_visualize = 1
print("Visualizing training images:")
visualize_images(train_loader, num_batches_to_visualize)

print("Visualizing testing images:")
visualize_images(test_loader, num_batches_to_visualize)